In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import cicada.communicator

@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    print("Hello, World!")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57246 from tcp://127.0.0.1:57246.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57246 from tcp://127.0.0.1:57247.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.


Hello, World!Hello, World!



INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [3]:
import cicada.logging

@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.info("Hello, World!")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57263 from tcp://127.0.0.1:57263.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57263 from tcp://127.0.0.1:57264.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Hello, World!
INFO:root:Hello, World!
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [4]:
@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank}!")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57279 from tcp://127.0.0.1:57279.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57279 from tcp://127.0.0.1:57280.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Hello from player 0!
INFO:root:Hello from player 1!
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [5]:
@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank} of {communicator.world_size}!")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57294 from tcp://127.0.0.1:57294.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57294 from tcp://127.0.0.1:57295.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Hello from player 0 of 2!
INFO:root:Hello from player 1 of 2!
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [6]:
import cicada.encoder

encoder = cicada.encoder.FixedFieldEncoder()
encoder

cicada.encoder.FixedFieldEncoder(modulus=18446744073709551557, precision=16)

In [7]:
import numpy

value = numpy.array(numpy.pi)
value

array(3.14159265)

In [8]:
encoded_value = encoder.encode(value)
encoded_value

array(205887, dtype=object)

The encoder turns the unencoded array of real values into an array of integers with the same shape that encode the original values.  205887 may seem like an unlikely way to store the value of $\pi$, so let's decode it and see if the decoded value matches the original:

In [9]:
decoded_value = encoder.decode(encoded_value)
decoded_value

array(3.1415863)

You can see that the result is a value that's *close* to the original, but not an exact match.  This is because the default 16 bits of precision used by FixedFieldEncoder to represent fractions can only approximate some values (of course, the original value was also a finite approximation of $\pi$, so this shouldn't bother you too much).  For many computations 16 bits of fractional precision is more than enough, but if you need more (or less) precision, you can create an encoder with custom parameters.  For example, if we double the number of fractional bits:

In [10]:
encoder = cicada.encoder.FixedFieldEncoder(precision=32)
encoder.decode(encoder.encode(value))

array(3.14159265)

... we get a much closer match to the original value.

In [11]:
import cicada.additive

@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator=communicator, seed=1234)
    
    secret = numpy.array(numpy.pi) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")
    
    share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    log.info(f"Player {communicator.rank} share: {share}")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57309 from tcp://127.0.0.1:57309.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57309 from tcp://127.0.0.1:57310.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret: 3.141592653589793
INFO:root:Player 1 secret: None
INFO:root:Player 0 share: cicada.additive.AdditiveArrayShare(storage=16993927038499601873)
INFO:root:Player 1 share: cicada.additive.AdditiveArrayShare(storage=1452817035210155571)
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [12]:
@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator=communicator, seed=1234)
    
    secret = numpy.array(numpy.pi) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")
    
    share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    log.info(f"Player {communicator.rank} share: {share}")

    revealed = protocol.encoder.decode(protocol.reveal(share))
    log.info(f"Player {communicator.rank} revealed: {revealed}")
    
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57326 from tcp://127.0.0.1:57326.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57326 from tcp://127.0.0.1:57327.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret: 3.141592653589793
INFO:root:Player 1 secret: None
INFO:root:Player 0 share: cicada.additive.AdditiveArrayShare(storage=16993927038499601873)
INFO:root:Player 1 share: cicada.additive.AdditiveArrayShare(storage=1452817035210155571)
INFO:root:Player 0 revealed: 3.1415863037109375
INFO:root:Player 1 revealed: 3.1415863037109375
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [13]:
@cicada.communicator.NNGCommunicator.run(world_size=2)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator=communicator, seed=1234)
    
    if communicator.rank == 0:
        fortune = numpy.array(10000000)
    elif communicator.rank == 1:
        fortune = numpy.array(12000000)
    log.info(f"Player {communicator.rank} fortune: {fortune}")
 
    fortune = protocol.encoder.encode(fortune)

    share0 = protocol.share(src=0, secret=fortune, shape=())
    share1 = protocol.share(src=1, secret=fortune, shape=())

    winner = 0
    winning_share = share0
    
    if not protocol.reveal(protocol.less(share1, winning_share)):
        winner = 1
        winning_share = share1
    
    log.info(f"Player {communicator.rank} winner: {winner}")
    
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57345 from tcp://127.0.0.1:57345.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57345 from tcp://127.0.0.1:57346.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 fortune: 10000000
INFO:root:Player 1 fortune: 12000000
INFO:root:Player 0 winner: 1
INFO:root:Player 1 winner: 1
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None


[None, None]

In [14]:
@cicada.communicator.NNGCommunicator.run(world_size=4)
def main(communicator):
    log = cicada.logging.Logger(logger=logging.getLogger(), communicator=communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator=communicator, seed=1234)

    fortune = numpy.loadtxt(f"../examples/millionaire-{communicator.rank}.txt")

    winner = None
    winning_share = protocol.share(src=0, secret=protocol.encoder.zeros(shape=()), shape=())
    for rank in communicator.ranks:
        fortune_share = protocol.share(src=rank, secret=protocol.encoder.encode(fortune), shape=())
        less_share = protocol.less(fortune_share, winning_share)
        less = protocol.reveal(less_share)
        if not less:
            winner = rank
            winning_share = fortune_share

    log.info(f"Player {communicator.rank} winner: {winner}")
    
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57362 from tcp://127.0.0.1:57362.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57362 from tcp://127.0.0.1:57363.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:57362 from tcp://127.0.0.1:57364.
INFO:cicada.communicator.nng:Player 3 rendezvous with tcp://127.0.0.1:57362 from tcp://127.0.0.1:57365.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 3 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 winner: 2
INFO:root:Player 1 winner: 2
INFO:root:Player 2 winner: 2
INFO:root:Player 3 winner: 2
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' 

[None, None, None, None]

If we examine the contents of the individual players' files, we see that the choice of winner is correct:

In [15]:
fortunes = []
for rank in range(4):
    fortunes.append(numpy.loadtxt(f"../examples/millionaire-{rank}.txt"))

for rank, fortune in enumerate(fortunes):
    print(f"Player {rank} fortune: {fortune:>10}")

Player 0 fortune: 10000000.0
Player 1 fortune:  9000000.0
Player 2 fortune: 11000000.0
Player 3 fortune:  8500000.0
